In [5]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
from tabulate import tabulate
import numpy as np
from pandas import Series, DataFrame
import json
import urllib
import requests, zipfile, io

# Method creates a dataframe from the data scraped from web page

In [7]:
def sec_dataframe():
    data = [];
    File_URL = [];
    date = [];
    type = [];
    i = 0;
    count = 0;
    container = [];
    smaller_container = [];


    url = "https://www.sec.gov/foia/iareports/inva-archive.htm";
    page = urllib.request.urlopen(url);

    soup = BeautifulSoup(page, 'html.parser');

    mydivs = soup.find_all("div", {"class": "grid_7 alpha"});

    for list in soup.find_all("div", {"class": "grid_7 alpha"}):
        for li in list.find_all('a'):
            container.append(li.get('href'));

    for i, val in enumerate(container):
        if (i >= 130):
            smaller_container.append(val);
        else:
            if (i % 2 == 0):
                smaller_container.append(val)

    dataframe = pd.DataFrame(smaller_container, columns=['File_URL']);
    dataframe = "https://www.sec.gov" + dataframe

    link = "https://www.sec.gov"

    for lis in soup.find_all("div", {"class": "grid_7 alpha"}):
        for li in lis.find_all('li'):
            data.append(li.get_text());

    for j, date_container in enumerate(data):
        if (date_container.endswith('(Exempt Reporting Advisers)')):
            date.append(date_container[:-28]);
            type.append("Exempt");
        else:
            date.append(date_container);
            type.append("Non-Exempt");

    dataframe['Date'] = date
    dataframe['Type'] = type

    return dataframe

In [79]:
periods = ["February 2010"]
is_exempt = False
only_most_recent = False;

# Finding the link of the selected file

In [64]:
# Params :  
# periods : Time Period
# is_exempt : Exemption
# only_most_recent : Find for recent time period

In [80]:
def get_sec_zip_by_period(periods, is_exempt, only_most_recent):
    dataframe = sec_dataframe()
    if is_exempt == False and only_most_recent == False:
        link = dataframe.loc[(dataframe['Date'] == periods[0]) & (dataframe['Type'] == 'Non-Exempt')]
        return link
    elif is_exempt == True and only_most_recent == False:
        link = dataframe.loc[(dataframe['Date'] == periods[0]) & (dataframe['Type'] == 'Exempt')]
    elif is_exempt == True and only_most_recent == True:
        for date in dataframe['date']:
            if date == 'June 2017' or date == 'May 2017' or date == 'April 2017' or date == 'March 2017' or date == 'Feburary 2017' or date == 'January 2017':
                link = dataframe.loc[((dataframe['Date'] == date) & (dataframe['Type'] == 'Exempt'))]
    elif is_exempt == False and only_most_recent == True:
        for date in dataframe['date']:
            if date == 'June 2017' or date == 'May 2017' or date == 'April 2017' or date == 'March 2017' or date == 'Feburary 2017' or date == 'January 2017':
                link = dataframe.loc[((dataframe['Date'] == date) & (dataframe['Type'] == 'Non-Exempt'))]
                
                 
    return link

In [81]:
link = get_sec_zip_by_period(periods,is_exempt,only_most_recent)
pageLink = link['File_URL']
pageLink

88    https://www.sec.gov/foia/iareports/ia020110.zip
Name: File_URL, dtype: object

In [82]:
pageLink = "".join(pageLink)

In [83]:
# Link for the zip file

In [84]:
pageLink

'https://www.sec.gov/foia/iareports/ia020110.zip'

In [85]:
r = requests.get(pageLink)
z = zipfile.ZipFile(io.BytesIO(r.content))

In [86]:
name = z.namelist()
name

['IA FOIA Download 2-1-10.CSV']

In [87]:
last=name[0][-1]
last

'V'

In [88]:
name = "".join(name)
name

'IA FOIA Download 2-1-10.CSV'

In [89]:
# Extracting the file

In [90]:
z.extractall("/home/sarthak/Downloads")

In [91]:
if last == 'V':
    df = pd.read_csv("/home/sarthak/Downloads/"+name, encoding = "ISO-8859-1")
else:
    df = pd.ExcelFile("/home/sarthak/Downloads/"+name)
    

In [92]:
# DataFrame created from the file inside the zip file

In [93]:
df

,SEC Region,Organization CRD#,SEC#,Primary Business Name,Legal Name,Main Office Street Address 1,Main Office Street Address 2,Main Office City,Main Office State,Main Office Country,...,11E(1),11E(2),11E(3),11E(4),11F,11G,11H(1)(a),11H(1)(b),11H(1)(c),11H(2)
0,CHRO,7365,801-39365,"WRP INVESTMENTS, INC.","WRP INVESTMENTS, INC.",4407 BELMONT AVE,NaN,YOUNGSTOWN,OH,USA,...,N,N,N,N,N,N,Y,N,N,Y
1,ARO,108741,801-34451,"SOUTHERNSUN ASSET MANAGEMENT, INC","SOUTHERNSUN ASSET MANAGEMENT, INC",6000 POPLAR AVENUE,SUITE 220,MEMPHIS,TN,USA,...,N,N,N,N,N,N,N,N,N,N
2,SFRO,114093,801-62709,LIGHTHOUSE FINANCIAL PLANNING,"LIGHTHOUSE FINANCIAL PLANNING, LLC","101 PARK SHORE DRIVE, SUITE 100",NaN,FOLSOM,CA,USA,...,N,N,N,N,N,N,N,N,N,N
3,BRO,150778,801-70371,TRADEX CAPITAL MARKETS,"TRADEX CAPITAL MARKETS, LLC.",35 MASON STREET,NaN,GREENWICH,CT,UNITED STATES,...,N,N,N,N,N,N,N,N,N,N
4,LARO,150220,801-70152,"ARMORED WOLF, LLC","ARMORED WOLF, LLC",65 ENTERPRISE,NaN,ALISO VIEJO,CA,UNITED STATES,...,N,N,N,N,N,N,N,N,N,N
5,SFRO,127283,801-63997,"ADDISON AVENUE FINANCIAL PARTNERS, LLC","ADDISON AVENUE FINANCIAL PARTNERS, LLC",1011 SUNSET BLVD.,NaN,ROCKLIN,CA,UNITED STATES,...,N,N,N,N,N,N,N,N,N,N
6,ARO,113364,801-60356,"SOUTHPORT CAPITAL, INC.",LIVINGSTON GROUP ASSET MANAGEMENT COMPANY,ONE UNION SQUARE SUITE 300,NaN,CHATTANOOGA,TN,UNITED STATES,...,N,N,N,N,N,N,N,N,N,N
7,LARO,133260,801-63757,"CONWAY CAPITAL MANAGEMENT, INC.","CONWAY CAPITAL MANAGEMENT, INC.",11560 ACAMA STREET,NaN,STUDIO CITY,CA,UNITED STATES,...,N,N,N,N,N,N,N,N,N,N
8,DRO,128667,801-62453,DIVIDEND CAPITAL INVESTMENTS LLC,DIVIDEND CAPITAL INVESTMENTS LLC,518 17TH STREET,17TH FLOOR,DENVER,CO,UNITED STATES,...,N,N,N,N,N,N,N,N,N,N
9,NYRO,148499,801-69896,INTEGRATED WEALTH MANAGEMENT,INTEGRATED WEALTH MANAGEMENT,RARITAN PLAZA 111,105 FIELDCREST AVE STE 201,EDISON,NJ,USA,...,N,N,N,N,N,N,N,N,N,N
